In [1]:
import numpy as np

In [2]:
%load_ext Cython

In [3]:
# Simple example.
X = np.asarray([[0,1,1,0]])
Z = np.asarray([3,2])

# Expected output.
Y_expected = np.asarray([[3,2,2,3]])

In [4]:
def python_loop(X,H,W,Z):
    Y = np.zeros(shape=X.shape, dtype=np.int)
    for h in range(H):
        for w in range(W):
            val = X[h,w]
            Y[h,w] = Z[val]
    return Y

In [5]:
Y = python_loop(X,H=X.shape[0],W=X.shape[1],Z=Z)

In [6]:
Y

array([[3, 2, 2, 3]])

In [7]:
assert np.alltrue(Y_expected == Y)

In [8]:
# Bigger example.
H = 5000
W = 5000
X = np.random.randint(6, size=(H,W))
Z = np.asarray([10,11,12,13,14,15])

In [9]:
X

array([[0, 1, 0, ..., 3, 5, 5],
       [2, 1, 5, ..., 4, 0, 2],
       [4, 3, 5, ..., 2, 4, 4],
       ..., 
       [4, 1, 5, ..., 4, 5, 4],
       [5, 0, 1, ..., 5, 4, 4],
       [2, 0, 2, ..., 3, 0, 0]])

In [10]:
Z

array([10, 11, 12, 13, 14, 15])

In [11]:
%%time
Y_loop = python_loop(X,H,W,Z)

CPU times: user 6.97 s, sys: 4 ms, total: 6.98 s
Wall time: 6.98 s


In [12]:
def vectorize_loop(X,Z):
    """A better? vectorized loop"""
    Y = np.zeros(shape=X.shape, dtype=np.int)
    for x, z in enumerate(Z):
        Y[X==x] = z
        
    return Y

In [13]:
%%time
Y_vec = vectorize_loop(X,Z)

CPU times: user 524 ms, sys: 16 ms, total: 540 ms
Wall time: 539 ms


In [14]:
assert np.alltrue(Y_vec == Y_vec)

In [15]:
%%cython
def cython_slow_loop(Y,X,H,W,Z):
    """Cython without any datatypes declared. Note the vars are pass-by-ref."""
    for h in range(H):
        for w in range(W):
            val = X[h,w]
            Y[h,w] = Z[val]
    return Y

In [16]:
%%time
Yc_slow = np.zeros(shape=X.shape, dtype=np.int)
cython_slow_loop(Yc_slow,X,H,W,Z)

CPU times: user 5.63 s, sys: 36 ms, total: 5.66 s
Wall time: 5.65 s


In [17]:
# Make sure values are equal.
assert np.alltrue(Yc_slow == Y_loop)

In [18]:
%%cython
def cython_loop(long [:,:] Y, long [:,:] X, long H, long W, long [:] Z):
    """Real fast cython loop with variable types declared."""
    cdef long h
    cdef long w
    cdef long val
    for h in range(H):
        for w in range(W):
            val = X[h,w]
            Y[h,w] = Z[val]
    return Y

In [19]:
%%time
Yc = np.zeros(shape=X.shape, dtype=np.int)
cython_loop(Yc,X,H,W,Z)

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 48.6 ms


In [20]:
assert np.alltrue(Yc == Y_loop)